# Data Wrangling!
Let's get the data prepared and ready to do some machine learning!

### Start by reading the data from the Lakehouse files!

In [1]:
df = (
    spark.read.option("header", True)
    .option("inferSchema", True)
    .csv("Files/churn/raw/churn.csv")
    .cache()
)

df.toPandas

display(df, summary=True)

StatementMeta(, fa91aa24-3927-460a-8b4a-8314c3e28520, 3, Finished, Available)

SynapseWidget(Synapse.DataFrame, 3e5b0c59-cbc8-4e51-9e24-c5ae1e6c79b5)

### Cleaning up the Data
Let's use the data wrangler to generate the code to:
- Drop duplicate RowNumbers and CustomerIds
- Drop missing data across all columns
- Drop undeeded rows, RowNumber, CustomerId, Surname

In [2]:
# Code generated by Data Wrangler for PySpark DataFrame

def clean_data(df):
    # Drop duplicate rows in columns: 'RowNumber', 'CustomerId'
    df = df.dropDuplicates(['RowNumber', 'CustomerId'])
    # Drop rows with missing data across all columns
    df = df.dropna()
    # Drop columns: 'RowNumber', 'CustomerId', 'Surname'
    df = df.drop('RowNumber', 'CustomerId', 'Surname')
    return df

df_clean = clean_data(df)
display(df_clean)

StatementMeta(, fa91aa24-3927-460a-8b4a-8314c3e28520, 48, Finished, Available)

SynapseWidget(Synapse.DataFrame, d3116d5b-198c-46ea-89ac-9546a3849979)

### One-hot Encode Categories
We will use the data wrangler to one-hot encode the categorical variables:
- Geography
- Gender

In [3]:
# Code generated by Data Wrangler for PySpark DataFrame

from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.functions import vector_to_array
from pyspark.sql import functions as F

def clean_data(df_clean):
    # One-hot encode columns: 'Geography', 'Gender'
    # ⚠️ This was generated to match the original pandas logic but may have performance issues.
    
    def one_hot_encode_col(df, key):
        indexer = StringIndexer(inputCol=key, outputCol='%s_numeric' % str(key), handleInvalid='keep')
        indexer_fitted = indexer.fit(df)
        df_indexed = indexer_fitted.transform(df)
        encoder = OneHotEncoder(inputCols=['%s_numeric' % str(key)], outputCols=['%s_onehot' % str(key)], dropLast=False)
        df_onehot = encoder.fit(df_indexed).transform(df_indexed)
        df_col_onehot = df_onehot.select('*', vector_to_array('%s_onehot' % str(key)).alias('%s_col_onehot' % str(key)))
        labels = sorted(indexer_fitted.labels)
        cols_expanded = [(F.col('%s_col_onehot' % str(key))[i].alias('%s_%s' % (str(key), labels[i]))) for i in range(len(labels))]
        df = df_col_onehot.select(*df.columns, *cols_expanded)
        df = df.drop(key)
        return df
    
    df_clean = one_hot_encode_col(df_clean, 'Geography')
    df_clean = one_hot_encode_col(df_clean, 'Gender')
    return df_clean

df_clean_1 = clean_data(df_clean)
display(df_clean_1)

StatementMeta(, fa91aa24-3927-460a-8b4a-8314c3e28520, 63, Finished, Available)

SynapseWidget(Synapse.DataFrame, 4452c284-8ef3-4aca-ab0a-05e6736d623b)

### Write the Cleaned Data to a Delta Table
Finally let's write our clean data to the Lakehouse as a delta table so we can use it for training a model!

In [4]:
table_name = "df_clean" 
df_clean_1.write.mode("overwrite").format("delta").save(f"Tables/{table_name}")
print(f"Spark dataframe saved to delta table: {table_name}")

StatementMeta(, fa91aa24-3927-460a-8b4a-8314c3e28520, 64, Finished, Available)

Spark dataframe saved to delta table: df_clean
